# LDA using Gensim

This notebook, goes through performing LDA on the text documents of the Climate Change PDFs using Gensim.

In this notebook, we will:
- identify the files and come up with a file list
- load the stopwords
- functionally get the documents
- normalize the documents
- create the dictionary and corpus
- create id2word
- initialize and train the model
- print out top topics
- determine the coherence score and perplexity
- create and save a visualization of topics using pyLDAvis
- test how an external document fits into the model

In [1]:
#imports
import codecs
import pandas as pd
import numpy as np
import os
import json
import time
import csv
import sklearn
import nltk
nltk.download('all')
import unicodedata
from nltk.corpus import PlaintextCorpusReader
from nltk.tokenize import WordPunctTokenizer
from nltk import pos_tag, sent_tokenize, wordpunct_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import CountVectorizer
from gensim.corpora import Dictionary
from sklearn.base import BaseEstimator, TransformerMixin
from pprint import pprint
from gensim.corpora import Dictionary
from gensim.models import LdaModel
from gensim.models.callbacks import PerplexityMetric
from gensim.models.callbacks import CoherenceMetric
from gensim.models.coherencemodel import CoherenceModel
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
import pyLDAvis.gensim
import pyLDAvis
from gensim.models import TfidfModel

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     /Users/rahimjiwa/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     /Users/rahimjiwa/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /Users/rahimjiwa/nltk_data...
[nltk_data]    |   Package biocreative_ppi is already up-to-date!
[nltk_data]    | Downloading package brown to
[nltk_data]    |     /Users/rahimjiwa/nltk_data...
[nltk_data]    |   Package brown is already up-to-date!
[nltk_data]    | Downloading package brown_tei to
[nltk_data]    |     /Users/rahimjiwa/nltk_data...
[nltk_data]    |   Package brown_tei is already up-to-date!
[nltk_data]    | Downloading package cess_cat to
[nltk_data]    |     /Users/rahimjiwa/nltk_data...
[nltk_data]    |   Package cess_cat is already up-

[nltk_data]    |   Package product_reviews_2 is already up-to-date!
[nltk_data]    | Downloading package pros_cons to
[nltk_data]    |     /Users/rahimjiwa/nltk_data...
[nltk_data]    |   Package pros_cons is already up-to-date!
[nltk_data]    | Downloading package qc to
[nltk_data]    |     /Users/rahimjiwa/nltk_data...
[nltk_data]    |   Package qc is already up-to-date!
[nltk_data]    | Downloading package reuters to
[nltk_data]    |     /Users/rahimjiwa/nltk_data...
[nltk_data]    |   Package reuters is already up-to-date!
[nltk_data]    | Downloading package rte to
[nltk_data]    |     /Users/rahimjiwa/nltk_data...
[nltk_data]    |   Package rte is already up-to-date!
[nltk_data]    | Downloading package semcor to
[nltk_data]    |     /Users/rahimjiwa/nltk_data...
[nltk_data]    |   Package semcor is already up-to-date!
[nltk_data]    | Downloading package senseval to
[nltk_data]    |     /Users/rahimjiwa/nltk_data...
[nltk_data]    |   Package senseval is already up-to-date!
[nlt

In [2]:
# let's start by running over the folder where our files are, we can use the plaintextcorpusreader to obtain the fileids
reader = PlaintextCorpusReader('tika_alldocs','.*txt')
# trying with added new docs

In [3]:
# create a list of fileids
file_list = reader.fileids()
file_list

['104890-2017_Climate_Change_Strategy.txt',
 '20170125-en.txt',
 '2017BernardSoubryPolicyBrief.txt',
 '81363.txt',
 "A Practitioners Guide to ClimateChange Adaptation in Ontario's Ecosystems Ver 1 2011.txt",
 'A_Canadian_Opportunity_-_Tackling_climate_change_by_switching_to_clean_power.txt',
 'A_Residential_Guide_to_Flood_Prevention_and_Recovery.txt',
 'Adapting to Climate Change in Coastal Communities In Canada White Paper.txt',
 'Adapting_to_Climate_Change_a_Risk_Based_Guide_for_Local_Governments_EN.txt',
 'Adapting_to_a_Changing_Climate_in_NS.txt',
 'Ahead-of-the-Storm-1.txt',
 'AtlanticRegionAdaptationScienceActivitiesReport.txt',
 'BC-Agriculture-Climate-Change-Action-Plan.txt',
 'Bruce_(2006)_AdaptingtoClimateChange_ARisk-basedGuideforONMunicipalities.txt',
 'CAT_research_plan_2015.txt',
 'CCCR_FULLREPORT-EN-FINAL.txt',
 'ClimatRisk-E-ACCESSIBLE.txt',
 'Climate Change Adaptation - A Priorities Plan for Canada (2012).txt',
 'Climate Change and Energy Sector.txt',
 'Climatecommunic

In [4]:
# load the stopwords text file
with open('stopwords.txt','r') as f:
    # create a list for stopwords
    stopwords = []
    # itearate through the text files, replacing any new line unicode characters
    for i in list(f):
        temp = str(i).replace('\n','')
        # append the stopwords to the list
        stopwords.append(temp)

In [5]:
print(stopwords)

['Stopwords and Punctuation:', '.', ',', 'and', 'the', 'of', 'to', '-', '/', 'in', '(', ')', 'a', 'for', ':', 'is', 'on', 'or', 'that', 'are', 'be', ').', '1', 'as', 'with', '://', '.,', 'from', '2', 'by', 'www', 'http', 'ca', '[', ']', '...', "'", '\\uf0b7', ';', 'at', ').', 'pdf', 'climate', 'change', 'canada', 'en', '):', 'so', 'html', 'do', '.;', 'com', 'canadian', 'impacts', 'adaptation']


In [6]:
# change directory to directory with files
os.chdir('/Users/rahimjiwa/Documents/DataScience/UofT3666_AppliedNLP/Final_Testings/tika_alldocs/')
# trying with added new docs

In [7]:
# create a list for documents
docs = []
# iterate through file list
for i in file_list:
    # load the documents
    text = PlaintextCorpusReader(root=os.getcwd(),fileids=i)
    # create a list of the words in the documents
    try:
        i_doc = list(text.words())
        # create a new list for document
        new_i_doc = []
        # for each word in the documents
        for j in i_doc:
            #check if the length of the word is greater than 2 and that it is not a stopword
            if len(j) > 2 and j.lower() not in stopwords:
                # append to word to the new list of words
                new_i_doc.append(j)
        # append list of words to documents
        docs.append(new_i_doc)
        # print the file name
        print(i)
    except ValueError:
        pass

104890-2017_Climate_Change_Strategy.txt
20170125-en.txt
2017BernardSoubryPolicyBrief.txt
81363.txt
A Practitioners Guide to ClimateChange Adaptation in Ontario's Ecosystems Ver 1 2011.txt
A_Canadian_Opportunity_-_Tackling_climate_change_by_switching_to_clean_power.txt
A_Residential_Guide_to_Flood_Prevention_and_Recovery.txt
Adapting to Climate Change in Coastal Communities In Canada White Paper.txt
Adapting_to_Climate_Change_a_Risk_Based_Guide_for_Local_Governments_EN.txt
Adapting_to_a_Changing_Climate_in_NS.txt
Ahead-of-the-Storm-1.txt
AtlanticRegionAdaptationScienceActivitiesReport.txt
BC-Agriculture-Climate-Change-Action-Plan.txt
CAT_research_plan_2015.txt
CCCR_FULLREPORT-EN-FINAL.txt
ClimatRisk-E-ACCESSIBLE.txt
Climate Change and Energy Sector.txt
ClimatecommunicationsintheNetherlands.txt
Coastal Flooding Ferryland, NL.txt
CoucilReport_july4_EN_Web.txt
En56-226-2008-eng.txt
Environment-and-Renewable-Energy-Workshop-Presentation-Abstracts-final.txt
FBC_WaterGuide_FINAL.txt
FloodReco

In [8]:
# take a look at the output of the documents
docs

[['Saskatchewan',
  'Prairie',
  'Resilience',
  'Made',
  'Saskatchewan',
  'Strategy',
  'Saskatchewan',
  'Saskatchewan',
  'people',
  'pragmatic',
  'resourceful',
  'innovative',
  'Throughout',
  'our',
  'history',
  'have',
  'faced',
  'complex',
  'challenging',
  'problems',
  'imposed',
  'geography',
  'Our',
  'population',
  'spread',
  'over',
  'vast',
  'land',
  'has',
  'taught',
  'self',
  'reliance',
  'resilience',
  'learned',
  'when',
  'faced',
  'challenge',
  'solve',
  'Today',
  'face',
  'global',
  'challenge',
  'once',
  'again',
  'our',
  'province',
  'motivated',
  'develop',
  'effective',
  'response',
  'Our',
  'industries',
  'heavily',
  'dependent',
  'fossil',
  'fuels',
  'produce',
  'energy',
  'food',
  'fertilizer',
  'products',
  'commodities',
  'needed',
  'around',
  'world',
  'come',
  'made',
  'Saskatchewan',
  'solutions',
  'encourage',
  'action',
  'meet',
  'challenges',
  'posed',
  'When',
  'faced',
  'complex',
  '

In [9]:
# check how many documents are being used
len(docs)

77

In [10]:
# define a normalizer class
class TextNormalizer(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.lemmatizer = WordNetLemmatizer()
    
    # define function for punctuation marks
    def is_punct(self, token):
        return all(unicodedata.category(char).startswith('P') for char in token)
    
    def normalize(self, document):
        return [
            # here you could return a lemmatized token instead
            self.lemmatize(token.lower())
            for sentence in document
            for token in sentence
            # check if a punctuation mark
            if not self.is_punct(token)
        ]
    
    # create a lemmatizer function
    def lemmatize(self, token):
        return self.lemmatizer.lemmatize(token)
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, documents):
        return [''.join(self.normalize(doc)) for doc in documents]

In [11]:
# initialize normalizer class
norm = TextNormalizer()

In [12]:
# create a list of the normalized documents
docs_normalized = []
# iterate through each document and apply the normalizer
for i in docs:
    docs_normalized.append(norm.fit_transform(i))

# take a look at the normalized documents
print(docs_normalized[:100])

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [13]:
# create a dictionary of the documents.
dictionary = Dictionary(docs_normalized)

# filter out words present in less than 3 documents or in more than 90% of the documents.
dictionary.filter_extremes(no_below=3, no_above=0.9)

2019-12-16 16:11:42,267 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2019-12-16 16:11:43,120 : INFO : built Dictionary(39554 unique tokens: ['000', '13462', '1980s', '1990', '1e13']...) from 77 documents (total 1135688 corpus positions)
2019-12-16 16:11:43,227 : INFO : discarding 27058 tokens: [('13462', 1), ('1e13', 1), ('5mt', 1), ('9485', 1), ('about', 74), ('all', 73), ('also', 73), ('areas', 71), ('backstop', 2), ('been', 70)]...
2019-12-16 16:11:43,228 : INFO : keeping 12496 tokens which were in no less than 3 and no more than 69 (=90.0%) documents
2019-12-16 16:11:43,258 : INFO : resulting dictionary: Dictionary(12496 unique tokens: ['000', '1980s', '1990', '2005', '2008']...)


In [14]:
# create a bow representation of the documents.
corpus = [dictionary.doc2bow(doc) for doc in docs_normalized]

In [15]:
# check the number of tokens and the number of documents
print('Number of unique tokens: %d' % len(dictionary))
print('Number of documents: %d' % len(corpus))

Number of unique tokens: 12496
Number of documents: 77


In [16]:
%%time
# index word dictionary
temp = dictionary[0]  
id2word = dictionary.id2token

top_terms = [5,10,15,20]
number_topics = [2,3,4,5,6,7,10]

for i in number_topics:
    for j in top_terms:
    # create the model
        model = LdaModel(
            corpus=corpus,
            id2word=id2word,
            alpha='auto',
            eta='auto',
            num_topics=i,
            eval_every=1,
            random_state=42
        )
        # find the top 10 words for each topic
        top_topics = model.top_topics(corpus, topn=j)
        # print the top 3 topics, and the top 10 words within those topics
        # also includes the topics coherence scores
        print('Number of topics: ' + str(i) + ' Number of Terms: ' + str(j))
        pprint(top_topics)
        # read a new file not trained to the model
        new_doc = PlaintextCorpusReader(root='/Users/rahimjiwa/Documents/DataScience/UofT3666_AppliedNLP/Final_Testings',fileids='random_pdf.txt')
        # get a list of words
        new_doc = list(new_doc.words())
        # create a bow representations of the new documents
        new_doc_bow = dictionary.doc2bow(new_doc)
        # get the document distributions
        print('Random Document Distribution')
        print(model.get_document_topics(new_doc_bow))
        # get the coherence score
        coherence_model = CoherenceModel(model=model, corpus=corpus, coherence='u_mass')
        print("Coherence: " + str(coherence_model.get_coherence()))
        # get the perplexity score
        print("Perplexity: " + str(model.log_perplexity(corpus)))

2019-12-16 16:11:43,885 : INFO : using autotuned alpha, starting with [0.5, 0.5]
2019-12-16 16:11:43,897 : INFO : using serial LDA version on this node
2019-12-16 16:11:43,928 : INFO : running online (single-pass) LDA training, 2 topics, 1 passes over the supplied corpus of 77 documents, updating model once every 77 documents, evaluating perplexity every 77 documents, iterating 50x with a convergence threshold of 0.001000
2019-12-16 16:11:43,930 : WARNING : too few updates, training might not converge; consider increasing the number of passes or iterations to improve accuracy
2019-12-16 16:11:45,681 : INFO : -9.811 per-word bound, 898.2 perplexity estimate based on a held-out corpus of 77 documents with 959067 words
2019-12-16 16:11:45,682 : INFO : PROGRESS: pass 0, at document #77/77
2019-12-16 16:11:45,991 : INFO : optimized alpha [0.869983, 0.91217685]
2019-12-16 16:11:45,996 : INFO : topic #0 (0.870): 0.004*"planning" + 0.004*"land" + 0.004*"energy" + 0.004*"development" + 0.003*"p

Number of topics: 2 Number of Terms: 5
[([(0.004147297, 'planning'),
   (0.003976375, 'land'),
   (0.003585681, 'energy'),
   (0.003559576, 'development'),
   (0.0034465855, 'plan')],
  -0.09536137343039641),
 ([(0.005554169, 'energy'),
   (0.0040934845, 'changes'),
   (0.0038522764, 'report'),
   (0.0037370163, 'emissions'),
   (0.0037185382, 'sea')],
  -0.19855591246416965)]
Random Document Distribution
[(0, 0.10143141), (1, 0.8985686)]
Coherence: -0.14222640404144926


2019-12-16 16:11:47,390 : INFO : -8.066 per-word bound, 267.9 perplexity estimate based on a held-out corpus of 77 documents with 959067 words
2019-12-16 16:11:47,391 : INFO : using autotuned alpha, starting with [0.5, 0.5]
2019-12-16 16:11:47,398 : INFO : using serial LDA version on this node
2019-12-16 16:11:47,418 : INFO : running online (single-pass) LDA training, 2 topics, 1 passes over the supplied corpus of 77 documents, updating model once every 77 documents, evaluating perplexity every 77 documents, iterating 50x with a convergence threshold of 0.001000
2019-12-16 16:11:47,421 : WARNING : too few updates, training might not converge; consider increasing the number of passes or iterations to improve accuracy


Perplexity: -8.065665893539961


2019-12-16 16:11:48,792 : INFO : -9.811 per-word bound, 898.2 perplexity estimate based on a held-out corpus of 77 documents with 959067 words
2019-12-16 16:11:48,794 : INFO : PROGRESS: pass 0, at document #77/77
2019-12-16 16:11:49,093 : INFO : optimized alpha [0.869983, 0.91217685]
2019-12-16 16:11:49,097 : INFO : topic #0 (0.870): 0.004*"planning" + 0.004*"land" + 0.004*"energy" + 0.004*"development" + 0.003*"plan" + 0.003*"management" + 0.003*"level" + 0.003*"sea" + 0.003*"community" + 0.003*"coastal"
2019-12-16 16:11:49,099 : INFO : topic #1 (0.912): 0.006*"energy" + 0.004*"changes" + 0.004*"report" + 0.004*"emissions" + 0.004*"sea" + 0.003*"future" + 0.003*"management" + 0.003*"level" + 0.003*"development" + 0.003*"land"
2019-12-16 16:11:49,100 : INFO : topic diff=1.522116, rho=1.000000


...

Perplexity: -8.194296510330643


2019-12-16 16:12:54,317 : INFO : -10.098 per-word bound, 1096.3 perplexity estimate based on a held-out corpus of 77 documents with 959067 words
2019-12-16 16:12:54,318 : INFO : PROGRESS: pass 0, at document #77/77
2019-12-16 16:12:54,609 : INFO : optimized alpha [0.18233542, 0.185698, 0.18769206, 0.18845245, 0.1848827, 0.18705593, 0.18017241, 0.18571228, 0.18557808, 0.18660522]
2019-12-16 16:12:54,622 : INFO : topic #6 (0.180): 0.005*"energy" + 0.004*"changes" + 0.004*"level" + 0.003*"development" + 0.003*"future" + 0.003*"planning" + 0.003*"emissions" + 0.003*"global" + 0.003*"sea" + 0.003*"land"
2019-12-16 16:12:54,623 : INFO : topic #0 (0.182): 0.004*"land" + 0.004*"development" + 0.004*"planning" + 0.004*"report" + 0.003*"energy" + 0.003*"transportation" + 0.003*"coastal" + 0.003*"plan" + 0.003*"risks" + 0.003*"emissions"
2019-12-16 16:12:54,625 : INFO : topic #5 (0.187): 0.006*"energy" + 0.005*"emissions" + 0.004*"development" + 0.004*"report" + 0.003*"changes" + 0.003*"governmen

Number of topics: 10 Number of Terms: 20
[([(0.0043884604, 'research'),
   (0.004029547, 'energy'),
   (0.0036049061, 'transportation'),
   (0.003591945, 'forest'),
   (0.003479897, 'changes'),
   (0.0034423464, 'government'),
   (0.0033330107, 'management'),
   (0.0032313599, 'coastal'),
   (0.0031589514, 'planning'),
   (0.0030093123, 'emissions'),
   (0.0029744778, 'report'),
   (0.002923624, 'sea'),
   (0.0028200778, 'infrastructure'),
   (0.0027928592, 'land'),
   (0.0027553109, 'community'),
   (0.002754537, 'level'),
   (0.002693396, 'future'),
   (0.002683115, 'assessment'),
   (0.0026585422, 'ice'),
   (0.0026392005, 'resources')],
  -0.12204075629009935),
 ([(0.0042568767, 'sea'),
   (0.0040918984, 'report'),
   (0.003735604, 'emissions'),
   (0.0036406913, 'energy'),
   (0.0035631943, 'risks'),
   (0.0034528512, 'development'),
   (0.003451777, 'infrastructure'),
   (0.003442626, 'management'),
   (0.0032847226, 'land'),
   (0.0030990536, 'changes'),
   (0.0030316547, 'futur

2019-12-16 16:12:55,740 : INFO : -8.194 per-word bound, 292.9 perplexity estimate based on a held-out corpus of 77 documents with 959067 words


Perplexity: -8.194296510330643
CPU times: user 2min 6s, sys: 2.43 s, total: 2min 8s
Wall time: 1min 11s


In [17]:
# create the model
model = LdaModel(
    corpus=corpus,
    id2word=id2word,
    alpha='auto',
    eta='auto',
    num_topics=3,
    eval_every=1,
    random_state=42
)

2019-12-16 16:12:55,750 : INFO : using autotuned alpha, starting with [0.33333334, 0.33333334, 0.33333334]
2019-12-16 16:12:55,759 : INFO : using serial LDA version on this node
2019-12-16 16:12:55,767 : INFO : running online (single-pass) LDA training, 3 topics, 1 passes over the supplied corpus of 77 documents, updating model once every 77 documents, evaluating perplexity every 77 documents, iterating 50x with a convergence threshold of 0.001000
2019-12-16 16:12:55,768 : WARNING : too few updates, training might not converge; consider increasing the number of passes or iterations to improve accuracy
2019-12-16 16:12:56,691 : INFO : -9.871 per-word bound, 936.2 perplexity estimate based on a held-out corpus of 77 documents with 959067 words
2019-12-16 16:12:56,691 : INFO : PROGRESS: pass 0, at document #77/77
2019-12-16 16:12:56,971 : INFO : optimized alpha [0.5889133, 0.6060606, 0.61193645]
2019-12-16 16:12:56,976 : INFO : topic #0 (0.589): 0.004*"land" + 0.004*"planning" + 0.004*"de

In [18]:
# find the top 10 words for each topic
top_topics = model.top_topics(corpus, topn=10)

In [19]:
# print the top 3 topics, and the top 10 words within those topics
# also includes the topics coherence scores
pprint(top_topics)

[([(0.004631676, 'energy'),
   (0.0045930427, 'management'),
   (0.0039982614, 'planning'),
   (0.0039370125, 'level'),
   (0.003597411, 'changes'),
   (0.0035456615, 'sea'),
   (0.0033018345, 'infrastructure'),
   (0.0031888986, 'future'),
   (0.003135145, 'land'),
   (0.0029807717, 'government')],
  -0.11312160203614283),
 ([(0.005660018, 'energy'),
   (0.0044247587, 'report'),
   (0.004079819, 'emissions'),
   (0.0040084627, 'changes'),
   (0.0036828027, 'sea'),
   (0.0036512404, 'coastal'),
   (0.003472614, 'development'),
   (0.003346625, 'land'),
   (0.003226643, 'future'),
   (0.0030616391, 'research')],
  -0.12724002054886982),
 ([(0.0041694823, 'land'),
   (0.0039146347, 'planning'),
   (0.003820342, 'development'),
   (0.0035005992, 'energy'),
   (0.0034255798, 'report'),
   (0.003383334, 'coastal'),
   (0.0032530655, 'risks'),
   (0.0031254895, 'emissions'),
   (0.0030975, 'plan'),
   (0.0030801585, 'sea')],
  -0.1981268823302916)]


In [20]:
# prepare inputs for the visualization
tfidf_model = TfidfModel(corpus)
corpus_tfidf = tfidf_model[corpus]

2019-12-16 16:12:57,088 : INFO : collecting document frequencies
2019-12-16 16:12:57,093 : INFO : PROGRESS: processing document #0
2019-12-16 16:12:57,165 : INFO : calculating IDF weights for 77 documents and 12496 features (158296 matrix non-zeros)


In [21]:
# initialize the visualization
viz = pyLDAvis.gensim.prepare(model, corpus_tfidf, dictionary)
# display the visualization
pyLDAvis.display(viz)

2019-12-16 16:12:59,392 : INFO : NumExpr defaulting to 4 threads.
/opt/anaconda3/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


In [22]:
# save the visualization.
pyLDAvis.save_html(viz, 'gensim_climate_change_lda_model_tika_alldocs.html')

In [23]:
# read a new file not trained to the model
new_doc = PlaintextCorpusReader(root='/Users/rahimjiwa/Documents/DataScience/UofT3666_AppliedNLP/Final_Testings',fileids='random_pdf.txt')
# get a list of words
new_doc = list(new_doc.words())
# create a bow representations of the new documents
new_doc_bow = dictionary.doc2bow(new_doc)
# get the document distributions
print(model.get_document_topics(new_doc_bow))

[(0, 0.058845174), (1, 0.66916853), (2, 0.2719863)]


In [24]:
# get the coherence score
coherence_model = CoherenceModel(model=model, corpus=corpus, coherence='u_mass')
print("Coherence: " + str(coherence_model.get_coherence()))


Coherence: -0.13066012318477496


In [25]:
# get the perplexity score
print("Perplexity: " + str(model.log_perplexity(corpus)))

2019-12-16 16:19:06,076 : INFO : -8.077 per-word bound, 270.1 perplexity estimate based on a held-out corpus of 77 documents with 959067 words


Perplexity: -8.077112493561172
